# Introduction to Clustering


## Supervised vs Unsupervised learning

Up until now we have been focusing on supervised learning. In supervised learning our training set consists of labeled data. For example, we have images and each image has an associated  label: dog, cat, elephant. And from this labeled data our model is trying to learn how to predict the label from the features. 

Unsupervised learning is trying to learn patterns from unlabeled data, and one set of models has to do with segmenting a dataset into clusters or groups of related data.

![](https://cambridge-intelligence.com/wp-content/uploads/2016/01/clustering-animated.gif)


In clustering the system divides a set of instances into clusters or groups
based on some measure of similarity. There are two main types of clustering algorithms.

### k-means clustering
For one type, we tell the algorithm how many clusters to make. Please cluster these 1,000
people into 5 groups. Please classify these web pages into 15 groups. These methods go by
the name of k-means clustering algorithms and we will discuss those a bit later.


### hierarchical clustering
For the other approach we don’t specify how many clusters to make. Instead the algorithm
starts with each instance in its own cluster. At each iteration of the algorithm it combines the
two most similar clusters into one. It repeatedly does this until there is only one cluster. This is called hierarchical clustering and its name makes sense. The running of the algorithm
results in one cluster, which consists of two sub-clusters. Each of those two sub-clusters in
turn, consist of 2 sub-sub clusters and so on. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/hierarchicalClustering1.png)


Again, at each iteration of the algorithm we join the two closest clusters. To determine the
‘closest clusters’ we use a distance formula. But we have some choices in how we compute
the distance between two clusters, which leads to different clustering methods. Consider the
three clusters (A, B, and C) illustrated below each containing two members. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/hierarchicalClustering2.png)




Which pair of
clusters should we join? Cluster A with B, or cluster C with B?

### Single-linkage clustering
In single-linkage clustering we define the distance between two clusters as the shortest
distance between any member of one cluster to any member of the other. With this
definition, the distance between Cluster A and Cluster B is the distance between A1 and B1,
since that is shorter than the distances between A1 and B2, A2 and B1, and A2 and B2. With
single-linkage clustering, Cluster A is closer to Cluster B than C is to B, so we would combine
A and B into a new cluster.

### Complete-linkage clustering
In complete-linkage clustering we define the distance between two clusters as the greatest
distance between any member of one cluster to any member of the other. With this
definition, the distance between Cluster A and Cluster B is the distance between A2 and B2.
With complete-linkage clustering, Cluster C is closer to Cluster B than A is to B, so we would
combine B and C into a new cluster.

### Average-linkage clustering
In average-linkage clustering we define the distance between two clusters as the average
distance between any member of one cluster to any member of the other. In the diagram
above, it appears that the average distance between Clusters C and B would be less than the
average between A and B and we would combine B and C into a new cluster. 



![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)


## Dog Breed Dataset

Let's explore this with a small dog breed dataset.

First, we will load the dataset:

In [ ]:
import pandas as pd
dog_data = pd.read_csv('https://raw.githubusercontent.com/zacharski/machine-learning-notebooks/master/data/dogbreeds.csv')
dog_data = dog_data.set_index('breed')

In [ ]:
dog_data

Looking at the values in the height and weight columns it looks like we should normalize the data.

<img src="http://animalfair.com/wp-content/uploads/2014/08/chihuahua-and-great-dane.jpg" width="700"/>





![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)


# <font color='#EE4C2C'>You Try ...</font> 

<font color='#EE4C2C'>1. Normalize the data</font> 

Normalize the data. Name the normalized dataset `dog_data`. HINT: This was covered in the sklearn lab.


In [ ]:

## TODO


And let's visualize that data:

In [ ]:
from matplotlib import pyplot as plt

# Set the figure size
plt.rcParams["figure.figsize"] = [14.00, 14]
plt.rcParams["figure.autolayout"] = True

# Scatter plot
ax = dog_data.plot.scatter(x='Height', y='Weight', alpha=0.5)

# Annotate each data point
for i, txt in enumerate(dog_data.index.values.tolist()):
   ax.annotate(txt, (dog_data.Height.iat[i]+0.02, dog_data.Weight.iat[i]))

plt.show()

Gazing at the scatter plot, it looks like we could group the data into three clusters. There are the 2 data points on the bottom left (*Chihuahua* and *Yorkshire Terrier*) The top right group of two (*Bull Mastiff* and *Great Dane*) and the middle group with all the other breeds. 

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)

##  Now let's compute the Euclidean distance between all the breeds:

This is just for pedagogical reasons --- no need to do this when using sklearn's clustering algorithms!


In [ ]:
from scipy.spatial.distance import pdist, squareform
pdist(dog_data)

pairwise = pd.DataFrame(
    squareform(pdist(dog_data)),
    columns = dog_data.index,
    index = dog_data.index
)

pairwise

### Hierarchical Clustering step by step

### Step 1.
Initially, each breed is in its own cluster. We find the two closest clusters and combine them
into one cluster. From the table above we see that the closest clusters are the
Border Collie and the Portuguese Water Dog (distance of 0.050470) so we combine them

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dendro1.png)

### Step 2. 
We find the two closest clusters and combine them into one cluster. From the table on the
preceding page we see that these are the Chihuahua and the Yorkshire Terrier (distance of
0.077200) so we combine them. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dendro2.png)

### Step 3.
We repeat the process again. This time combining the German Shepherd and the Golden
Retriever.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dendro3.png)

### Step 4.

We repeat the process yet again. From the table we see that the next closest pair is the Border
Collie and the Brittany Spaniel. The Border Collie is already in a cluster with the Portuguese
Water Dog which we created in Step 1. So in this step we are going to combine that cluster
with the Brittany Spaniel.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dendro4.png)

### Step 5.
And we continue ...

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dendro5.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/borderCollie.png)


## Hierarchical Clustering in sklearn

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/doodle.png)

Here is how to do hierarchical clustering in sklearn ...


In [ ]:
from sklearn.cluster import AgglomerativeClustering
clusterer = AgglomerativeClustering(affinity='euclidean', linkage='ward', n_clusters=3)
clusterer.fit_predict(dog_data)


And let's draw the dendrogram ...

In [ ]:
import scipy.cluster.hierarchy as sch
dendrogram = sch.dendrogram(sch.linkage(dog_data, method='single'), labels = dog_data.index, orientation = 'right')


### Let's disect this

Let's take a closer look at the parameters for AgglomerativeClustering `AgglomerativeClustering(affinity='euclidean', linkage='ward', n_clusters=3)`

`affinity='euclidean'` simply means to use the Euclidean distance. The options for affinity are `manhattan` and `cosine` among others. The default is euclidean.

`linkage='ward'` describes how we decide what clusters to join. The options are:


* `ward` minimizes the variance of the clusters being merged. (the default)
* `average` uses the average of the distances of each observation of the two sets.
* `complete` or ‘maximum’ linkage uses the maximum distances between all observations of the two sets.
* `single` uses the minimum of the distances between all observations of the two sets.

You'll notice that when we drew the dendrogram we used single linkage because it matched the explanation above. In general, it is good to use the default ward linkage.

`n_clusters=3` specifies how many clusters to create. In this case we instruct the algorithm to create 3 clusters. Let's do a scatter plot of those clusters:


In [ ]:
plt.rcParams["figure.figsize"] = [20.00, 16]
plt.rcParams["figure.autolayout"] = True
#ax = plt.scatter(x=dog_data['Height'], y=dog_data['Weight'], c= clusterer.labels_, cmap='rainbow' )
ax = dog_data.plot.scatter(x='Height', y='Weight', alpha=0.5, c= clusterer.labels_, cmap='rainbow', s=60)
for i, txt in enumerate(dog_data.index.values.tolist()):
   ax.annotate(txt, (dog_data.Height.iat[i]+0.02, dog_data.Weight.iat[i]))

plt.show()

So the Yorkshire Terrier and Chihuahua are in a cluster by themselves as is the Bullmastiff and the Great Dane. The remainder of the dog breeds are in the third cluster.



![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)

# k means clustering


![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/kmeans1.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/kmeans2.png)

The basic k-means algorithm is 

1. select k random instances to be the
initial centroids
2. REPEAT
3. assign each instance to the nearest
centroid. (forming k clusters)
4. update centroids by computing mean
of each cluster
5. UNTIL centroids don’t change (much).

Let’s go through an example. Consider the following points (x and y coordinates):

Let's divide our dog dataset into 3 clusters

In [ ]:
points = pd.DataFrame({'x':[1,1, 2, 2, 4, 4, 5, 5], 'y':[2,4,2,3,2,4,1,3]})
points

Say we want to cluster these into 2 groups.

**step 1 of above algorithm: select k random instances to be initial centroids.**

Suppose we randomly select (1, 4) as centroid 1 and (4, 2) as centroid 2.

**step 3: assign each instance to the nearest centroid**
To assign each instance to the nearest centroid we can use any of the distance measures we
have previously discussed. To keep things simple, for this example let’s use Manhattan
Distance. 

In [ ]:
points['CentOneDist'] = np.abs(points['x'] - 1) + np.abs(points['y'] - 4)
points['CentTwoDist'] = np.abs(points['x'] - 4) + np.abs(points['y'] - 2)
points

Based on these distances we assign the points to the following clusters:

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/cluster1.png)

step 4: update centroids
We compute the new centroids by computing the mean of each cluster. The mean x
coordinate of cluster 1 is:
(1 + 1 + 2) / 3 = 4/3 = 1.33
and the mean y is
(2 + 4 + 3) / 3 = 9/3 = 3
So the new cluster 1 centroid is (1.33, 3).
The new centroid for cluster 2 is (4, 2.4)

**step 5: until centroids don’t change**

The old centroids were (1, 4) and (4, 2) and the new ones are (1.33, 3) and (4, 2.4). The
centroids changed so we repeat.

**step 3: assign each instance to the nearest centroid**

Again we compute Manhattan Distance. 

In [ ]:
points['CentOneDist'] = np.abs(points['x'] - 1.33) + np.abs(points['y'] - 3)
points['CentTwoDist'] = np.abs(points['x'] - 4) + np.abs(points['y'] - 2.4)
points

and based on these distances assign the points to clusters:

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/cluster2.png)

**step 4: update centroids**

We compute the new centroids by computing the mean of each cluster.
* Cluster 1 centroid: (1.5, 2.75)
* Cluster 2 centroid: (4.5, 2.5)

**step 5: until centroids don’t change**
The centroids changed so we repeat.

**step 3: assign each instance to the nearest centroid**
Again we compute Manhattan Distance. 

In [ ]:
points['CentOneDist'] = np.abs(points['x'] - 1.5) + np.abs(points['y'] - 2.75)
points['CentTwoDist'] = np.abs(points['x'] - 4.5) + np.abs(points['y'] - 2.5)
points

and based on these distances assign the points to clusters:

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/cluster3.png)

**step 4: update centroids**

We compute the new centroids by computing the mean of each cluster.
Cluster 1 centroid: (1.5, 2.75)
Cluster 2 centroid: (4.5, 2.5)

**step 5: until centroids don’t change**

The updated centroids are identical to the previous ones so the algorithm converged on a
solution and we can stop. The final clusters are

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/cluster4.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/centroidText.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/centroidGood.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/CentroidSimple.png)

# Hill Climbing

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/Hill1.png)

I would like to briefly interrupt our
discussion of K-means clustering to talk
about hill climbing algorithms. Suppose our
goal is to reach the peak of some mountain
and we come up with the following
algorithm:

```
start at some random location on the mountain.
REPEAT
 take a step in the direction that will take you higher.
UNTIL there is no direction that will take you higher.
```

This seems like a reasonable algorithm.
Consider using it with the mountain shown here

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/Hill2.png)

You can see that regardless of where we are plopped
down on the mountain, we will reach the peak if we
follow the algorithm.
And if we think of this as a graph, we will reach the
peak value regardless of where we start on the graph.

Now let’s consider using the algorithm with the following graph

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/Hill3.png)

Thus, this simple version of the hill-climbing algorithm is not guaranteed to reach the
optimal solution.
![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/Hill4b.png)


![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/Hill5.png)


## SSE or Scatter

To determine the quality of a set of clusters we can use the sum of the squared error
(SSE). This is also called scatter. Here is how to compute it: for each point we will square
the distance from that point to its centroid, then add all those squared distances together.
More formally, 

$$SSE=\sum_{i=1}^{k}\sum_{x∈C_i}{dist(c_i,x)^2}$$

Let’s dissect that. In the first summation sign we are iterating over the clusters. So initially i
equals cluster 1, then i equals cluster 2, up to i equals cluster k. The next summation sign
iterates over the points in that cluster—something like, for each point x in cluster i. Dist is
whatever distance formula we are using (for example, Manhattan, or Euclidean). So we
compute the distance between that point, x, and the centroid for the cluster ci, square that
distance and add it to our total. 

Let’s say we run our k-means algorithm twice on the same data and for each run we pick a
different set of random initial centroids. Is the set of clusters that were computed during the
first run worse or better than the set computed during the second run? To answer that
question we compute the SSE for both sets of clusters. The set with the smaller SSE is the
better of the two.

# k-means in sklearn

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, ).fit(dog_data)
labels = kmeans.labels_

The variable `labels` is an array the specifies which group each dog belongs to:

In [ ]:
labels

My results were:

    array([0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 2], dtype=int32)

which indicates that the first, second, and third dogs are in group 0, the next one in group 1 and so on  That may be helpful for future computational tasks but is not the helpful if we are trying to visualize the data. Let me munge that a bit into a slightly more useful form:

In [ ]:
groups = {0: [], 1: [], 2: []}
i = 0
for index, row in dog_data.iterrows():
    groups[labels[i]].append(index)
    i += 1
## Now I will print it in a nice way:

for key, value in groups.items():
    print ('CLUSTER %i' % key)
    for breed in value:
        print("    %s" % breed)
    print('\n')

keep in mind that since they initial centroids are selected somewhat randomly it is possible that you get a different answer than I do. The answer I got was:

    CLUSTER 0
        Border Collie
        Boston Terrier
        Brittany Spaniel
        German Shepherd
        Golden Retreiver
        Portuguese Water Dog
        Standard Poodle
    

    CLUSTER 1
        Bullmastiff
        Great Dane


    CLUSTER 2
        Chihuahua
        Yorkshire Terrier




![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)


# <font color='#EE4C2C'>You Try ...</font> 
## <font color='#EE4C2C'>1. Breakfast Cereals</font> 

I would like you to create 4 clusters of the data in:

    https://raw.githubusercontent.com/zacharski/pg2dm-python/master/data/ch8/cereal.csv
    
For clustering use the features calories, sugar, protein, and fiber.

Print out the results as we did for the dog breed data:


    CLUSTER 0
    Bullmastiff
    Great Dane
    

    CLUSTER 1
        Chihuahua
        Yorkshire Terrier
    

    CLUSTER 2
        Border Collie
        Boston Terrier
        Brittany Spaniel
        German Shepherd
        Golden Retreiver
        Portuguese Water Dog
        Standard Poodle
        
Because the initial centroids are random, by default the sklearn kmeans agorithm runs the algorithm 10 times and picks the best results (based on some of squares error). I would like you to change that parameter so it runs the algorithm 100 times.   Just google `sklearn kmeans` to get documentation on the parameters.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>2. Hierarchical</font> 

I would like you to use the hierarchical clustering algorithm on the cereal data. Plot the dendrogram. 


In [ ]:
# TO DO

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)


# <font color='#EE4C2C'>Fighting Poverty</font> 
## <font color='#EE4C2C'>(bonus possible based on quality of work)</font> 



### Objective:
To categorise the countries using socio-economic and health factors that determine the overall development of the country.

### Problem Statement:
[HELP International's](https://help-international.org/) hmission is to fight poverty and empower people. Even though it has millions of dollars of funds, it is a tiny amount compared to the needs of the countries of the world. HELP needs to decide how to use this money strategically and effectively by selecting countries that are in the direst need of aid. Hence, your Job as a Data scientist is to categorise the countries using some socio-economic and health factors that determine the overall development of the country. Then you need to suggest the countries which the CEO needs to focus on the most.


Specifically, you need to

* determine what columns are relevant for this task 
* cluster the countries (you decide how many clusters)
* create a choropleth map showing the clusters
* determine the cluster representing countries of greatest need.
* list the countries with the greatest need.

Use whatever methods and hyperparameters you want. 

The [dataset](https://raw.githubusercontent.com/zacharski/ml-class/master/data/Country-data.csv) countains 10 measures for each of 167 countries:

| Column Name | Description |
| :--- | :--- |
country|Name of the country
child_mort|Death of children under 5 years of age per 1000 live births
exports|Exports of goods and services per capita. Given as %age of the GDP per capita
health|Total health spending per capita. Given as %age of GDP per capita
imports|Imports of goods and services per capita. Given as %age of the GDP per capita
Income|Net income per person
Inflation|The measurement of the annual growth rate of the Total GDP
life_expec|The average number of years a new born child would live if the current mortality patterns are to remain the same
total_fer|The number of children that would be born to each woman if the current age-fertility rates remain the same.
gdpp|The GDP per capita. Calculated as the Total GDP divided by the total population.

### Choropleth Map
A choropleth map allows us to visualize how a variable varies across a geographic area. There is a python library that helps us create one. 

Here is a quick example,


In [ ]:
countries = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/Country-data.csv')
countries[:10]

In [ ]:
import plotly.express as px

In [ ]:
fig = px.choropleth(countries,
                    locationmode='country names',
                    locations='country',
                    color='life_expec',
                    title='Coutries by Life Expectancy'
                   )
fig.show()